In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# OpenCV
import cv2

# Helper libraries
import os

In [2]:
test_dir = './all_frames/'
test_imgs = ['./all_frames/{}'.format(i) for i in os.listdir(test_dir)]  # get test images

In [3]:
nrows = 150
ncolumns = 150
channels = 3

In [4]:
# A function to read and process the images in an acceptable format for our model
def read_and_process_image(list_of_images):
    x = []
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (nrows, ncolumns), interpolation=cv2.INTER_CUBIC)) # Read the image
    
    return x

In [5]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

Using TensorFlow backend.


In [6]:
from keras.models import load_model
model = load_model('./model_keras.h5')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [7]:
from PIL import Image
import numpy as np
import re

test_datagen = ImageDataGenerator(rescale=1./255)
test_result_up = []

for i in range(len(test_imgs)):
    x_test = read_and_process_image(test_imgs[i:i+1])
    x = np.array(x_test)

    for batch in test_datagen.flow(x, batch_size=1):
        if model.predict(batch) < 0.5:
            frame_num = re.findall(r'\d+', test_imgs[i])[0]
            test_result_up.append(int(frame_num))
            
        break
        
    if(i%100==0 or i==(len(test_imgs)) - 1):
        print('{perc}% of process is completed.'.format(perc=i/len(test_imgs)*100))

0.0% of process is completed.
0.244798041616% of process is completed.
0.489596083231% of process is completed.
0.734394124847% of process is completed.
0.979192166463% of process is completed.
1.22399020808% of process is completed.
1.46878824969% of process is completed.
1.71358629131% of process is completed.
1.95838433293% of process is completed.
2.20318237454% of process is completed.
2.44798041616% of process is completed.
2.69277845777% of process is completed.
2.93757649939% of process is completed.
3.182374541% of process is completed.
3.42717258262% of process is completed.
3.67197062424% of process is completed.
3.91676866585% of process is completed.
4.16156670747% of process is completed.
4.40636474908% of process is completed.
4.6511627907% of process is completed.
4.89596083231% of process is completed.
5.14075887393% of process is completed.
5.38555691554% of process is completed.
5.63035495716% of process is completed.
5.87515299878% of process is completed.
6.1199510

50.4283965728% of process is completed.
50.6731946144% of process is completed.
50.9179926561% of process is completed.
51.1627906977% of process is completed.
51.4075887393% of process is completed.
51.6523867809% of process is completed.
51.8971848225% of process is completed.
52.1419828641% of process is completed.
52.3867809058% of process is completed.
52.6315789474% of process is completed.
52.876376989% of process is completed.
53.1211750306% of process is completed.
53.3659730722% of process is completed.
53.6107711138% of process is completed.
53.8555691554% of process is completed.
54.1003671971% of process is completed.
54.3451652387% of process is completed.
54.5899632803% of process is completed.
54.8347613219% of process is completed.
55.0795593635% of process is completed.
55.3243574051% of process is completed.
55.5691554468% of process is completed.
55.8139534884% of process is completed.
56.05875153% of process is completed.
56.3035495716% of process is completed.
56.

In [8]:
test_result_up.sort()

In [9]:
segmentation = []
frame_counting = False

frame_batch = []
for frame in test_result_up:
    if not frame_counting:
        frame_counting = True
        frame_batch.append(frame)
        
    else:
        diff_last = frame - frame_batch[len(frame_batch) - 1]
        
        if diff_last > 5:
            segmentation.append(frame_batch)
            frame_batch = []
            frame_batch.append(frame)
            
        else:
            frame_batch.append(frame)

In [11]:
import shutil

video_output_dir = './segmentation_output/'
if os.path.exists(video_output_dir):
    shutil.rmtree(video_output_dir, ignore_errors = True)
    
os.mkdir(video_output_dir)

for segment in segmentation:
    frame_arr = []
    
    for frame in segment:
        frame_file = test_dir + 'frame' + str(frame) + '.jpg'
        frame_img = cv2.imread(frame_file)
        frame_arr.append(frame_img)
        
    height, width, layers = frame_arr[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')
    size = (width, height)
    fps = 30
    
    video_path = video_output_dir + 'video_' + str(segment[0]) + '_' + str(segment[len(frame_arr) - 1]) + '.mp4'
    
    cap = cv2.VideoWriter(video_path, fourcc, fps, size)
    for i in range(len(frame_arr)):
        cap.write(frame_arr[i])
        
    cap.release()